#  3. Building Functions and Pipeline (pre MLOps work) of the Preproc work

### Highlights of this Notebook:

<p style="font-size:14px">
- This notebook it was build to organize the preproc work for deploying; <br>
- All the preprocessing of the Dataset (cleaning, encoding, etc) it is organized in 6 functions wrapped in another one; <br>
</p>

✅ Previus concluded: model discovery and tunning, used this here on this notebook

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
# Checking whether a numerical feature has a normal distribution or not
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import cross_validate, learning_curve, train_test_split, StratifiedKFold, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, classification_report, precision_recall_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import randint
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

In [2]:
pd.set_option('display.max_columns', None)

### 3.1 Using the raw data to confirm the applicability of the functions createded.

In [3]:
url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/train.csv'
df = pd.read_csv(url)
df.head(4)

/tmp/ipykernel_105888/471918335.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good


In [4]:
data = df.copy()
data.head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.82262,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.94496,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good


In [5]:
data.shape

(100000, 28)

In [6]:
data['Payment_Behaviour'].value_counts(ascending=False)

Payment_Behaviour
Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
!@9#%8                               7600
Name: count, dtype: int64

In [7]:
#DEF Removing y and customer id - only if needed
def remove_ids(df: pd.DataFrame):
    df = df.drop(["Customer_ID", "Credit_Score"], axis=1)
    print("Customer Id and y: DONE ✅")
    return df

### 3.2 Creating Functions

#### 3.2.1 Data Clearning Function
<p style="font-size:14px">
- It removes useless features, drop duplicates and handle with special characters <br>
- It applies 'nan' in rows to handle after; <br>

In [8]:
# DEF DATA CLEANING: dropping  useless features, duplicates and cleaning special characters with nan

def cleaning(df: pd.DataFrame):
    #dropping useless features:
    df = df.drop(['ID', 'Name', 'SSN', 'Monthly_Inhand_Salary', 'Type_of_Loan'], axis=1)
    #dropping duplicates
    df = df.drop_duplicates()
    #removing _ for empty
    df = df.replace('_', '', regex=True)
    #removing empty for nan
    df.replace('', np.nan, inplace=True)
    #Identify rows with empty values
    rows_with_empty_values = df[df.eq('').any(axis=1)]
    #Fill empty values with NaN in selected rows
    df.loc[rows_with_empty_values.index] = df.loc[rows_with_empty_values.index].replace('', np.nan)
    #cleaning special characters
    df.replace('!@9#%8', np.nan, inplace=True)
    #Communicating
    print("Data Cleaning: DONE ✅")
    #Returning df
    return df

In [9]:
#Applying the function
data = cleaning(data)

Data Cleaning: DONE ✅


In [10]:
#Simple sanity check
data['Payment_Behaviour'].value_counts(ascending=False)

Payment_Behaviour
LowspentSmallvaluepayments      25513
HighspentMediumvaluepayments    17540
LowspentMediumvaluepayments     13861
HighspentLargevaluepayments     13721
HighspentSmallvaluepayments     11340
LowspentLargevaluepayments      10425
Name: count, dtype: int64

#### 3.2.2 Cronological Function
<p style="font-size:14px">
- Encode and transform month in sin and cos; <br>
- Encode cronological; <br>

In [11]:
#Handling date and cronological features (enconding AND hadling with missing values)

def cronological(df: pd.DataFrame):
    months_in_year = 12
    dic_date = {"January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
                "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12}

    df['Month'] = df['Month'].map(dic_date)

    df["sin_Month"] = np.sin(2*np.pi*df[["Month"]] / months_in_year)
    df["cos_Month"] = np.cos(2*np.pi*df[["Month"]] / months_in_year)

    # Extract years and months from 'Credit_History_Age' column
    df['Credit_History_Years'] = df['Credit_History_Age'].str.extract(r'(\d+)\s*Years?').astype(float)
    df['Credit_History_Months'] = df['Credit_History_Age'].str.extract(r'(\d+)\s*Months?').astype(float)

    # Handle missing or NaN values
    df['Credit_History_Years'] = df['Credit_History_Years'].fillna(0)
    df['Credit_History_Months'] = df['Credit_History_Months'].fillna(0)

    # Convert years to months and add with months to get total months
    df['Credit_History_Age_Months'] = (df['Credit_History_Years'] * 12 + df['Credit_History_Months']).astype(int)

    # Drop the intermediate columns used for calculation (optional, if you no longer need them)
    df.drop(columns=['Credit_History_Years', 'Credit_History_Months'], inplace=True)

    # Display counts of unique values in 'Credit_History_Age_Months'
    value_counts = df['Credit_History_Age_Months'].value_counts()

    # Replace 0 with NaN in the 'Credit_History_Age_Months' column
    df['Credit_History_Age_Months'] = df['Credit_History_Age_Months'].replace(0, np.nan)

    # Group by 'Customer_ID' and calculate the mean for 'Credit_History_Age_Months' within each group
    mean_by_Customer_ID = df.groupby('Customer_ID')['Credit_History_Age_Months'].transform(lambda x: x.mean())

    # Fill NaN values in 'Credit_History_Age' with the mean for each group
    df['Credit_History_Age_Months'] = df['Credit_History_Age_Months'].fillna(mean_by_Customer_ID)

    df.drop(columns=['Credit_History_Age', 'Month'], inplace=True)

    print("Cronological features: DONE ✅")
    return df

#### 3.2.3 Missing Function
<p style="font-size:14px">
- Hadle with missing values using mode for categorical or mean for numeric features; <br>

In [12]:
#aqui é o tratamento dos missing (otimizado), tirei history age e coloquei uma função chamada cronological

def missing(df: pd.DataFrame):
    list_mode = ["Credit_Mix", "Occupation", "Payment_Behaviour"]
    list_mean = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]

    for feature in list_mean:
        df[feature] = pd.to_numeric(df[feature], errors='coerce')

    for feature in list_mode:
        mode_by_Customer_ID = df.groupby('Customer_ID')[feature].transform(lambda x: x.mode()[0])
        df[feature] = df[feature].fillna(mode_by_Customer_ID)

    for feature in list_mean:
        mean_by_Customer_ID = df.groupby('Customer_ID')[feature].transform(lambda x: x.mean())
        df[feature] = df[feature].fillna(mean_by_Customer_ID)

    print("Data Missing: DONE ✅")
    return df

In [13]:
# Esse codigo é do missing não otimizado.
# def missing(df: pd.DataFrame):
#     list_to_num = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]
#     list_mode = ["Credit_Mix", "Occupation",  ]
#     list_mean = ["Num_of_Delayed_Payment", "Amount_invested_monthly", "Changed_Credit_Limit", "Num_Credit_Inquiries", "Monthly_Balance"]

#     #Credit Mix
#     mode_by_Customer_ID = df.groupby('Customer_ID')['Credit_Mix'].transform(lambda x: x.mode()[0])
#     df['Credit_Mix'] = df['Credit_Mix'].fillna(mode_by_Customer_ID)

#     #Occupation
#     mode_by_Customer_ID = df.groupby('Customer_ID')['Occupation'].transform(lambda x: x.mode()[0])
#     df['Occupation'] = df['Occupation'].fillna(mode_by_Customer_ID)

#     #num_of_delay_payments
#     df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.mean())
#     df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].fillna(mean_by_Customer_ID)

#     #Amount_invested_monthly
#     df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.mean())
#     df['Amount_invested_monthly'] = df['Amount_invested_monthly'].fillna(mean_by_Customer_ID)

#     #Changed_Credit_Limit
#     df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: x.mean())
#     df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(mean_by_Customer_ID)

#     #Num_Credit_Inquiries
#     df['Num_Credit_Inquiries'] = pd.to_numeric(df['Num_Credit_Inquiries'], errors='coerce')
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.mean())
#     df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].fillna(mean_by_Customer_ID)

#     #Monthly_Balance
#     df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.mean())
#     df['Monthly_Balance'] = df['Monthly_Balance'].fillna(mean_by_Customer_ID)

#     print("Data Missing DONE ✅")
#     return df


#### 3.2.4 Outliers
<p style="font-size:14px">
- Uses measures of statistical dispersion to handle with outliers: 3 x Interquartile Range (Q3 - Q1); <br>
- Uses "business infos" to handle with "strange" numbers; <br>

In [14]:
#Outliers
def outliers(df: pd.DataFrame):

    #Age
    df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
    df.loc[(df['Age'] > 100) | (df['Age'] < 0), 'Age'] = np.nan
    mode_by_Customer_ID = df.groupby('Customer_ID')['Age'].transform(lambda x: x.mode()[0])
    df['Age'] = df['Age'].fillna(mode_by_Customer_ID)

    #Num bank account
    df['Num_Bank_Accounts'] = pd.to_numeric(df['Num_Bank_Accounts'], errors='coerce')
    df.loc[df['Num_Bank_Accounts']<0, 'Num_Bank_Accounts'] = 0  # Replacing account balances less than zero with zero
    Q1 = float(df['Num_Bank_Accounts'].quantile(0.25))
    Q3 = float(df['Num_Bank_Accounts'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Num_Bank_Accounts'] > Outlier) | (df['Num_Bank_Accounts'] < 0), 'Num_Bank_Accounts'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Bank_Accounts'].transform(lambda x: x.mean())
    df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].fillna(mean_by_Customer_ID)

    #Credit Card
    df['Num_Credit_Card'] = pd.to_numeric(df['Num_Credit_Card'], errors='coerce')
    Q1 = float(df['Num_Credit_Card'].quantile(0.25))
    Q3 = float(df['Num_Credit_Card'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Num_Credit_Card'] > Outlier) | (df['Num_Credit_Card'] < 0), 'Num_Credit_Card'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Card'].transform(lambda x: x.mean())
    df['Num_Credit_Card'] = df['Num_Credit_Card'].fillna(mean_by_Customer_ID)

    #Interest rate
    df['Interest_Rate'] = pd.to_numeric(df['Interest_Rate'], errors='coerce')
    Q1 = float(df['Interest_Rate'].quantile(0.25))
    Q3 = float(df['Interest_Rate'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Interest_Rate'] > Outlier) | (df['Interest_Rate'] < 0), 'Interest_Rate'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Interest_Rate'].transform(lambda x: x.mean())
    df['Interest_Rate'] = df['Interest_Rate'].fillna(mean_by_Customer_ID)

    #_Num_of_Loan_
    df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')
    df.loc[df['Num_of_Loan']<0, 'Num_of_Loan'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Num_of_Loan'].quantile(0.25))
    Q3 = float(df['Num_of_Loan'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Num_of_Loan'] > Outlier) | (df['Num_of_Loan'] < 0), 'Num_of_Loan'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Loan'].transform(lambda x: x.mean())
    df['Num_of_Loan'] = df['Num_of_Loan'].fillna(mean_by_Customer_ID)

    #Delay_from_due_date
    df['Delay_from_due_date'] = pd.to_numeric(df['Delay_from_due_date'], errors='coerce')
    df.loc[df['Delay_from_due_date']<0, 'Delay_from_due_date'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Delay_from_due_date'].quantile(0.25))
    Q3 = float(df['Delay_from_due_date'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Delay_from_due_date'] > Outlier) | (df['Delay_from_due_date'] < 0), 'Delay_from_due_date'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Delay_from_due_date'].transform(lambda x: x.mean())
    df['Delay_from_due_date'] = df['Delay_from_due_date'].fillna(mean_by_Customer_ID)

    #Number of delay payment
    df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
    df.loc[df['Num_of_Delayed_Payment']<0, 'Num_of_Delayed_Payment'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Num_of_Delayed_Payment'].quantile(0.25))
    Q3 = float(df['Num_of_Delayed_Payment'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Num_of_Delayed_Payment'] > Outlier) | (df['Num_of_Delayed_Payment'] < 0), 'Num_of_Delayed_Payment'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.mean())
    df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].fillna(mean_by_Customer_ID)

    #Changed credit limit
    df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
    df.loc[df['Changed_Credit_Limit']<0, 'Changed_Credit_Limit'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Changed_Credit_Limit'].quantile(0.25))
    Q3 = float(df['Changed_Credit_Limit'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Changed_Credit_Limit'] > Outlier) | (df['Changed_Credit_Limit'] < 0), 'Changed_Credit_Limit'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: x.mean())
    df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(mean_by_Customer_ID)

    #Num_Credit_Inquiries
    df['Num_Credit_Inquiries'] = pd.to_numeric(df['Num_Credit_Inquiries'], errors='coerce')
    df.loc[df['Num_Credit_Inquiries']<0, 'Num_Credit_Inquiries'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Num_Credit_Inquiries'].quantile(0.25))
    Q3 = float(df['Num_Credit_Inquiries'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Num_Credit_Inquiries'] > Outlier) | (df['Num_Credit_Inquiries'] < 0), 'Num_Credit_Inquiries'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.mean())
    df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].fillna(mean_by_Customer_ID)

    #Outstanding debt
    df['Outstanding_Debt'] = pd.to_numeric(df['Outstanding_Debt'], errors='coerce')
    df.loc[df['Outstanding_Debt']<0, 'Outstanding_Debt'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Outstanding_Debt'].quantile(0.25))
    Q3 = float(df['Outstanding_Debt'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Outstanding_Debt'] > Outlier) | (df['Outstanding_Debt'] < 0), 'Outstanding_Debt'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Outstanding_Debt'].transform(lambda x: x.mean())
    df['Outstanding_Debt'] = df['Outstanding_Debt'].fillna(mean_by_Customer_ID)

    #Credit_Utilization_Ratio
    df['Credit_Utilization_Ratio'] = pd.to_numeric(df['Credit_Utilization_Ratio'], errors='coerce')
    df.loc[df['Credit_Utilization_Ratio']<0, 'Credit_Utilization_Ratio'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Credit_Utilization_Ratio'].quantile(0.25))
    Q3 = float(df['Credit_Utilization_Ratio'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Credit_Utilization_Ratio'] > Outlier) | (df['Credit_Utilization_Ratio'] < 0), 'Credit_Utilization_Ratio'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Credit_Utilization_Ratio'].transform(lambda x: x.mean())
    df['Credit_Utilization_Ratio'] = df['Credit_Utilization_Ratio'].fillna(mean_by_Customer_ID)

    #Total_EMI_per_month
    df['Total_EMI_per_month'] = pd.to_numeric(df['Total_EMI_per_month'], errors='coerce')
    df.loc[df['Total_EMI_per_month']<0, 'Total_EMI_per_month'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Total_EMI_per_month'].quantile(0.25))
    Q3 = float(df['Total_EMI_per_month'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Total_EMI_per_month'] > Outlier) | (df['Total_EMI_per_month'] < 0), 'Total_EMI_per_month'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Total_EMI_per_month'].transform(lambda x: x.mean())
    df['Total_EMI_per_month'] = df['Total_EMI_per_month'].fillna(mean_by_Customer_ID)

    #Amount_invested_monthly
    df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
    df.loc[df['Amount_invested_monthly']<0, 'Amount_invested_monthly'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Amount_invested_monthly'].quantile(0.25))
    Q3 = float(df['Amount_invested_monthly'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Amount_invested_monthly'] > Outlier) | (df['Amount_invested_monthly'] < 0), 'Amount_invested_monthly'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.mean())
    df['Amount_invested_monthly'] = df['Amount_invested_monthly'].fillna(mean_by_Customer_ID)

    #Payment behauviour coloquei no missing values (pq estou suponto que vai tratar o special characters 1o)

    #Monthly_Balance
    df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
    df.loc[df['Monthly_Balance']<0, 'Monthly_Balance'] = 0  # Replacing less than zero with zero
    Q1 = float(df['Monthly_Balance'].quantile(0.25))
    Q3 = float(df['Monthly_Balance'].quantile(0.75))
    IQR = Q3 - Q1
    Outlier = Q3 + 3 * IQR
    df.loc[(df['Monthly_Balance'] > Outlier) | (df['Monthly_Balance'] < 0), 'Monthly_Balance'] = np.nan
    mean_by_Customer_ID = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.mean())
    df['Monthly_Balance'] = df['Monthly_Balance'].fillna(mean_by_Customer_ID)

    #Credit_History_Age_Months
    df['Credit_History_Age_Months'] = pd.to_numeric(df['Credit_History_Age_Months'], errors='coerce')
    df.loc[df['Credit_History_Age_Months']<0, 'Credit_History_Age_Months'] = 0  # Replacing less than zero with zero

    print("Outliers: DONE ✅")
    return df


In [15]:
# #AQUI É SÓ UMA COPIA DA PARTE DE CIMA, ATENÇÃO
# def outliers(df: pd.DataFrame):

#     #Age
#     df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
#     df.loc[(df['Age'] > 100) | (df['Age'] < 0), 'Age'] = np.nan
#     mode_by_Customer_ID = df.groupby('Customer_ID')['Age'].transform(lambda x: x.mode()[0])
#     df['Age'] = df['Age'].fillna(mode_by_Customer_ID)

#     #Num bank account
#     df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].astype(float)
#     df.loc[df['Num_Bank_Accounts']<0, 'Num_Bank_Accounts'] = 0  # Replacing account balances less than zero with zero
#     Q1 = 3
#     Q3 = 7
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Num_Bank_Accounts'] = pd.to_numeric(df['Num_Bank_Accounts'], errors='coerce')
#     df.loc[(df['Num_Bank_Accounts'] > Outlier) | (df['Num_Bank_Accounts'] < 0), 'Num_Bank_Accounts'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Bank_Accounts'].transform(lambda x: x.mean())
#     df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].fillna(mean_by_Customer_ID)

#     #Credit Card
#     Q1 = 4
#     Q3 = 7
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Num_Credit_Card'] = pd.to_numeric(df['Num_Credit_Card'], errors='coerce')
#     df.loc[(df['Num_Credit_Card'] > Outlier) | (df['Num_Credit_Card'] < 0), 'Num_Credit_Card'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Card'].transform(lambda x: x.mean())
#     df['Num_Credit_Card'] = df['Num_Credit_Card'].fillna(mean_by_Customer_ID)

#     #Interest rate
#     Q1 = 8
#     Q3 = 20
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Interest_Rate'] = pd.to_numeric(df['Interest_Rate'], errors='coerce')
#     df.loc[(df['Interest_Rate'] > Outlier) | (df['Interest_Rate'] < 0), 'Interest_Rate'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Interest_Rate'].transform(lambda x: x.mean())
#     df['Interest_Rate'] = df['Interest_Rate'].fillna(mean_by_Customer_ID)

#     #_Num_of_Loan_
#     df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')
#     df.loc[df['Num_of_Loan']<0, 'Num_of_Loan'] = 0  # Replacing less than zero with zero
#     Q1 = 1
#     Q3 = 5
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')
#     df.loc[(df['Num_of_Loan'] > Outlier) | (df['Num_of_Loan'] < 0), 'Num_of_Loan'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Loan'].transform(lambda x: x.mean())
#     df['Num_of_Loan'] = df['Num_of_Loan'].fillna(mean_by_Customer_ID)

#     #Delay_from_due_date
#     df['Delay_from_due_date'] = pd.to_numeric(df['Delay_from_due_date'], errors='coerce')
#     df.loc[df['Delay_from_due_date']<0, 'Delay_from_due_date'] = 0  # Replacing less than zero with zero
#     Q1 = 10
#     Q3 = 28
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Delay_from_due_date'] = pd.to_numeric(df['Delay_from_due_date'], errors='coerce')
#     df.loc[(df['Delay_from_due_date'] > Outlier) | (df['Delay_from_due_date'] < 0), 'Delay_from_due_date'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Delay_from_due_date'].transform(lambda x: x.mean())
#     df['Delay_from_due_date'] = df['Delay_from_due_date'].fillna(mean_by_Customer_ID)

#     #Number of delay payment
#     df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
#     df.loc[df['Num_of_Delayed_Payment']<0, 'Num_of_Delayed_Payment'] = 0  # Replacing less than zero with zero
#     Q1 = 9
#     Q3 = 18
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')
#     df.loc[(df['Num_of_Delayed_Payment'] > Outlier) | (df['Num_of_Delayed_Payment'] < 0), 'Num_of_Delayed_Payment'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.mean())
#     df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].fillna(mean_by_Customer_ID)

#     #Changed credit limit
#     df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
#     df.loc[df['Changed_Credit_Limit']<0, 'Changed_Credit_Limit'] = 0  # Replacing less than zero with zero
#     Q1 = 5.34
#     Q3 = 14.85
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
#     df.loc[(df['Changed_Credit_Limit'] > Outlier) | (df['Changed_Credit_Limit'] < 0), 'Changed_Credit_Limit'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Changed_Credit_Limit'].transform(lambda x: x.mean())
#     df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(mean_by_Customer_ID)

#     #Num_Credit_Inquiries
#     df['Num_Credit_Inquiries'] = pd.to_numeric(df['Num_Credit_Inquiries'], errors='coerce')
#     df.loc[df['Num_Credit_Inquiries']<0, 'Num_Credit_Inquiries'] = 0  # Replacing less than zero with zero
#     Q1 = 3
#     Q3 = 9
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Num_Credit_Inquiries'] = pd.to_numeric(df['Num_Credit_Inquiries'], errors='coerce')
#     df.loc[(df['Num_Credit_Inquiries'] > Outlier) | (df['Num_Credit_Inquiries'] < 0), 'Num_Credit_Inquiries'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.mean())
#     df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].fillna(mean_by_Customer_ID)

#     #Outstanding debt
#     df['Outstanding_Debt'] = pd.to_numeric(df['Outstanding_Debt'], errors='coerce')
#     df.loc[df['Outstanding_Debt']<0, 'Outstanding_Debt'] = 0  # Replacing less than zero with zero
#     Q1 = 566.072500
#     Q3 = 1945.962500
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Outstanding_Debt'] = pd.to_numeric(df['Outstanding_Debt'], errors='coerce')
#     df.loc[(df['Outstanding_Debt'] > Outlier) | (df['Outstanding_Debt'] < 0), 'Outstanding_Debt'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Outstanding_Debt'].transform(lambda x: x.mean())
#     df['Outstanding_Debt'] = df['Outstanding_Debt'].fillna(mean_by_Customer_ID)

#     #Credit_Utilization_Ratio
#     df['Credit_Utilization_Ratio'] = pd.to_numeric(df['Credit_Utilization_Ratio'], errors='coerce')
#     df.loc[df['Credit_Utilization_Ratio']<0, 'Credit_Utilization_Ratio'] = 0  # Replacing less than zero with zero
#     Q1 = 28.052567
#     Q3 = 36.496663
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Credit_Utilization_Ratio'] = pd.to_numeric(df['Credit_Utilization_Ratio'], errors='coerce')
#     df.loc[(df['Credit_Utilization_Ratio'] > Outlier) | (df['Credit_Utilization_Ratio'] < 0), 'Credit_Utilization_Ratio'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Credit_Utilization_Ratio'].transform(lambda x: x.mean())
#     df['Credit_Utilization_Ratio'] = df['Credit_Utilization_Ratio'].fillna(mean_by_Customer_ID)

#     #Total_EMI_per_month
#     df['Total_EMI_per_month'] = pd.to_numeric(df['Total_EMI_per_month'], errors='coerce')
#     df.loc[df['Total_EMI_per_month']<0, 'Total_EMI_per_month'] = 0  # Replacing less than zero with zero
#     Q1 = 30.306660
#     Q3 = 161.224249
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Total_EMI_per_month'] = pd.to_numeric(df['Total_EMI_per_month'], errors='coerce')
#     df.loc[(df['Total_EMI_per_month'] > Outlier) | (df['Total_EMI_per_month'] < 0), 'Total_EMI_per_month'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Total_EMI_per_month'].transform(lambda x: x.mean())
#     df['Total_EMI_per_month'] = df['Total_EMI_per_month'].fillna(mean_by_Customer_ID)

#     #Amount_invested_monthly
#     df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
#     df.loc[df['Amount_invested_monthly']<0, 'Amount_invested_monthly'] = 0  # Replacing less than zero with zero
#     Q1 = 76.057014
#     Q3 = 274.671232
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')
#     df.loc[(df['Amount_invested_monthly'] > Outlier) | (df['Amount_invested_monthly'] < 0), 'Amount_invested_monthly'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.mean())
#     df['Amount_invested_monthly'] = df['Amount_invested_monthly'].fillna(mean_by_Customer_ID)

#     #Payment behauviour coloquei no missing values (pq estou suponto que vai tratar o special characters 1o)

#     #Monthly_Balance
#     df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
#     df.loc[df['Monthly_Balance']<0, 'Monthly_Balance'] = 0  # Replacing less than zero with zero
#     Q1 = 267.992189
#     Q3 = 467.673454
#     IQR = Q3 - Q1
#     Outlier = Q3 + 3 * IQR
#     df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')
#     df.loc[(df['Monthly_Balance'] > Outlier) | (df['Monthly_Balance'] < 0), 'Monthly_Balance'] = np.nan
#     mean_by_Customer_ID = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.mean())
#     df['Monthly_Balance'] = df['Monthly_Balance'].fillna(mean_by_Customer_ID)

#     #Credit_History_Age_Months
#     df['Credit_History_Age_Months'] = pd.to_numeric(df['Credit_History_Age_Months'], errors='coerce')
#     df.loc[df['Credit_History_Age_Months']<0, 'Credit_History_Age_Months'] = 0  # Replacing less than zero with zero

#     print("Outliers DONE ✅")
#     return df


#### 3.2.5 Scaling
<p style="font-size:14px">
- Uses Robust Scaler to handle with all numerical; <br>
- Uses K-Nearest Neighbors (KNN) to scale "Total_EMI_per_month" ; <br>

In [16]:
# Scaling

def scaling_num(df: pd.DataFrame):
    #numerical_features = df.select_dtypes(exclude=['object'])
    numerical_features = df[["sin_Month",
                             "cos_Month",
                             "Age",
                             "Annual_Income",
                             "Num_Bank_Accounts",
                             "Num_Credit_Card",
                             "Interest_Rate",
                             "Num_of_Loan",
                             "Delay_from_due_date",
                             "Num_of_Delayed_Payment",
                             "Changed_Credit_Limit",
                             "Num_Credit_Inquiries",
                             "Outstanding_Debt",
                             "Credit_Utilization_Ratio",
                             "Credit_History_Age_Months",
                             "Total_EMI_per_month",
                             "Amount_invested_monthly",
                             "Monthly_Balance"]]

    imputer = KNNImputer(n_neighbors=2)
    imputed_data = imputer.fit_transform(numerical_features)
    numerical_features = pd.DataFrame(imputed_data, columns=numerical_features.columns)

    robust_scaler = RobustScaler().set_output(transform='pandas')
    numerical_features_scaled = robust_scaler.fit_transform(numerical_features)

    df[numerical_features.columns] = numerical_features_scaled

    print("Scaling Numerical: DONE ✅")
    return df

#### 3.2.5 Encoding
<p style="font-size:14px">
- One Hot Encoding for categorical; <br>

In [17]:
#Encoding categorical (just categorical, not necessary the numerical)

def encoding(df: pd.DataFrame):
    # Selecionando as features categóricas
    #categorical_features = df.select_dtypes(include=['object'])
    categorical_features = df[["Occupation", "Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour"]]

    # Instanciando e ajustando o OneHotEncoder
    ohe = OneHotEncoder(drop="if_binary", sparse_output=False, min_frequency=0.05).set_output(transform='pandas')
    encoded_features = ohe.fit_transform(categorical_features)

    # Adicionando as colunas codificadas ao DataFrame original
    df = pd.concat([df, encoded_features], axis=1)  # Concatenando as novas colunas codificadas
    df = df.drop(columns=categorical_features.columns)  # Removendo as colunas categóricas originais

    print("Encoding Categorical: DONE ✅")

    return df

### 3.3 Applying the functions in a pipeline

In [18]:
def pipeline(df: pd.DataFrame):
    functions = [cleaning, cronological, missing, outliers, scaling_num, encoding]
    for func in functions:
        df = func(df)
    return df

In [19]:
df_cleaned = pipeline(df)
df_cleaned.head(4)

Data Cleaning: DONE ✅
Cronological features: DONE ✅
Data Missing: DONE ✅
Outliers: DONE ✅
Scaling Numerical: DONE ✅
Encoding Categorical: DONE ✅


,Customer_ID,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score,sin_Month,cos_Month,Credit_History_Age_Months,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_HighspentLargevaluepayments,Payment_Behaviour_HighspentMediumvaluepayments,Payment_Behaviour_HighspentSmallvaluepayments,Payment_Behaviour_LowspentLargevaluepayments,Payment_Behaviour_LowspentMediumvaluepayments,Payment_Behaviour_LowspentSmallvaluepayments
0,CUS0xd40,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.833333,-0.777778,0.196635,-0.2,-0.258118,-0.649349,-0.142087,-0.310616,-0.122178,Good,0.000000,1.378396e+00,0.291755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,CUS0xd40,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-1.000000,-0.888889,0.196635,-0.2,-0.258118,-0.042731,-0.142087,-0.078987,-0.261204,Good,0.369340,1.009056e+00,0.312896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,CUS0xd40,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.833333,-0.777778,0.091483,-0.2,-0.258118,-0.437753,-0.142087,-0.302760,-0.028800,Good,0.504528,5.045279e-01,0.304440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CUS0xd40,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.722222,-1.111111,-0.329127,-0.2,-0.258118,-0.109890,-0.142087,0.417597,-0.566438,Good,0.369340,3.360831e-16,0.310782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 3.4 Removing Customer_ID and y for model

In [20]:
df_for_model = df_cleaned.drop(["Customer_ID", "Credit_Score"], axis=1)
df_for_model.head(4)

,Age,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,sin_Month,cos_Month,Credit_History_Age_Months,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,Credit_Mix_Bad,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_NM,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_HighspentLargevaluepayments,Payment_Behaviour_HighspentMediumvaluepayments,Payment_Behaviour_HighspentSmallvaluepayments,Payment_Behaviour_LowspentLargevaluepayments,Payment_Behaviour_LowspentMediumvaluepayments,Payment_Behaviour_LowspentSmallvaluepayments
0,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.833333,-0.777778,0.196635,-0.2,-0.258118,-0.649349,-0.142087,-0.310616,-0.122178,0.000000,1.378396e+00,0.291755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-1.000000,-0.888889,0.196635,-0.2,-0.258118,-0.042731,-0.142087,-0.078987,-0.261204,0.369340,1.009056e+00,0.312896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.833333,-0.777778,0.091483,-0.2,-0.258118,-0.437753,-0.142087,-0.302760,-0.028800,0.504528,5.045279e-01,0.304440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.555556,-0.346209,-0.5,-0.333333,-0.769231,0.25,-0.722222,-1.111111,-0.329127,-0.2,-0.258118,-0.109890,-0.142087,0.417597,-0.566438,0.369340,3.360831e-16,0.310782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Backup: double check the infos 

### Backup: Interrompido temporariamente pois preciso aguardar Greg fazer as edições na base dele

In [21]:
# url_greg = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/df_preprocessed_20240825.csv'
# df_greg = pd.read_csv(url_greg)
# df_greg.head(4)

In [22]:
# if df_cleaned.sort_index().reset_index(drop=True).equals(df_greg.sort_index().reset_index(drop=True)):
#     print("All good!")
# else:
#     print("Double check, go, you can do this")

In [23]:
# # Verificar se os DataFrames têm o mesmo índice e colunas
# assert df_cleaned.columns.equals(df_greg.columns)
# assert df_cleaned.index.equals(df_greg.index)

# # Verificar onde os DataFrames são diferentes
# difference = df_cleaned.compare(df_greg)

# print("Diferenças encontradas:")
# print(difference)

In [24]:
# comparison = df_cleaned.compare(df_greg)

# print("Diferenças encontradas:")
# print(comparison)

In [25]:
# if df_cleaned.shape != df_greg.shape:
#     print("DataFrames têm formatos diferentes.")
# else:
#     print("ok, iguais")

In [26]:
# df_cleaned = df_cleaned.reindex_like(df_greg)
# df_greg = df_greg.reindex_like(df_cleaned)

In [27]:
# df_greg.isnull().sum().sort_values(ascending=False)

In [28]:
df_cleaned.isnull().sum().sort_values(ascending=False)

Customer_ID                                       0
Age                                               0
Annual_Income                                     0
Num_Bank_Accounts                                 0
Num_Credit_Card                                   0
Interest_Rate                                     0
Num_of_Loan                                       0
Delay_from_due_date                               0
Num_of_Delayed_Payment                            0
Changed_Credit_Limit                              0
Num_Credit_Inquiries                              0
Outstanding_Debt                                  0
Credit_Utilization_Ratio                          0
Total_EMI_per_month                               0
Amount_invested_monthly                           0
Monthly_Balance                                   0
Credit_Score                                      0
sin_Month                                         0
cos_Month                                         0
Credit_Histo

In [29]:
df_cleaned["Annual_Income"]

0       -0.346209
1       -0.346209
2       -0.346209
3       -0.346209
4       -0.346209
           ...   
99995    0.038445
99996    0.038445
99997    0.038445
99998    0.038445
99999    0.038445
Name: Annual_Income, Length: 100000, dtype: float64

In [30]:
# df_greg["Annual_Income"]

In [31]:
# comparison = df_cleaned == df_greg


In [32]:
# comparison

In [33]:
# comparison = df_cleaned.compare(df_greg)

# print("Diferenças encontradas:")
# print(comparison)

In [34]:
# comparison

In [35]:
# df_greg.shape

In [36]:
# df_greg.columns

In [37]:
# df_cleaned.columns

In [38]:
df_cleaned.shape

(100000, 47)

#### Backup: Just checking if the TEST set in Kaggle has the same behaviour with the TRAIN set (edit: confirmed) 

In [39]:
#Averiguando como está a base de TESTE
url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/test.csv'
df_test = pd.read_csv(url)
df_test.head(2)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196


In [40]:
df_test.shape

(50000, 27)

In [41]:
df_test.isna().sum().sort_values(ascending=False)

Monthly_Inhand_Salary       7498
Type_of_Loan                5704
Name                        5015
Credit_History_Age          4470
Num_of_Delayed_Payment      3498
Amount_invested_monthly     2271
Num_Credit_Inquiries        1035
Monthly_Balance              562
Customer_ID                    0
Month                          0
ID                             0
Num_Credit_Card                0
Num_Bank_Accounts              0
Annual_Income                  0
Occupation                     0
Age                            0
SSN                            0
Delay_from_due_date            0
Interest_Rate                  0
Credit_Mix                     0
Changed_Credit_Limit           0
Num_of_Loan                    0
Outstanding_Debt               0
Payment_of_Min_Amount          0
Credit_Utilization_Ratio       0
Total_EMI_per_month            0
Payment_Behaviour              0
dtype: int64

### OK! The Test Dataset behaviors the same with the Train Dataset.

## Below the pipeline for deploy

### Enconding , Scaling and cronological inside a pipeline

In [42]:
num_preproc = Pipeline([
    ("scaler", RobustScaler())
])

In [43]:
cat_preproc = Pipeline([
    ("ohe", OneHotEncoder(drop="if_binary", sparse_output=False, min_frequency=0.05))
])

In [44]:
preproc = ColumnTransformer([
    ("num_tr", num_preproc, ["sin_Month",
                             "cos_Month",
                             "Age",
                             "Annual_Income",
                             "Num_Bank_Accounts",
                             "Num_Credit_Card",
                             "Interest_Rate",
                             "Num_of_Loan",
                             "Delay_from_due_date",
                             "Num_of_Delayed_Payment",
                             "Changed_Credit_Limit",
                             "Num_Credit_Inquiries",
                             "Outstanding_Debt",
                             "Credit_Utilization_Ratio",
                             "Credit_History_Age_Months",
                             "Total_EMI_per_month",
                             "Amount_invested_monthly",
                             "Monthly_Balance"]),
    ("cat_tr", cat_preproc, ["Occupation", "Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour"])
])

In [45]:
url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/train.csv'
df = pd.read_csv(url)
df.head(3)

/tmp/ipykernel_105888/1267714592.py:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good


In [46]:
data = df.copy()

In [47]:
data = cleaning(df=data)
data = cronological(df=data)
data = missing(df=data)
data = outliers(df=data)
data.head(2)

Data Cleaning: DONE ✅
Cronological features: DONE ✅
Data Missing: DONE ✅
Outliers: DONE ✅


,Customer_ID,Age,Occupation,Annual_Income,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,sin_Month,cos_Month,Credit_History_Age_Months
0,CUS0xd40,23.0,Scientist,19114.12,3.0,4.0,3.0,4.0,3.0,7.0,11.27,4.0,Good,809.98,26.82262,No,49.574949,80.415295,HighspentSmallvaluepayments,312.494089,Good,0.500000,0.866025,265.000000
1,CUS0xd40,23.0,Scientist,19114.12,3.0,4.0,3.0,4.0,0.0,6.0,11.27,4.0,Good,809.98,31.94496,No,49.574949,118.280222,LowspentLargevaluepayments,284.629162,Good,0.866025,0.500000,268.333333


In [48]:
X = data.drop(["Credit_Score", "Customer_ID"], axis=1)
y = data["Credit_Score"]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [50]:
#Clean
#Missing
#outlier
#chamar o cronologial
#pega a DF split train e test
#pega o pipe e fita (encoding e scaler)
# e depois exporta o pipe pra arquivo
# ler o arquivo na main e lá na main fazer o cronological e depois aplicar o pipe

In [51]:
preproc_deploy = preproc.fit(X_train)
preproc_deploy

ColumnTransformer(transformers=[('num_tr',
                                 Pipeline(steps=[('scaler', RobustScaler())]),
                                 ['sin_Month', 'cos_Month', 'Age',
                                  'Annual_Income', 'Num_Bank_Accounts',
                                  'Num_Credit_Card', 'Interest_Rate',
                                  'Num_of_Loan', 'Delay_from_due_date',
                                  'Num_of_Delayed_Payment',
                                  'Changed_Credit_Limit',
                                  'Num_Credit_Inquiries', 'Outstanding_Debt',
                                  'Credit_Utilization_Ratio',
                                  'Credit_History_Age_Months',
                                  'Total_EMI_per_month',
                                  'Amount_invested_monthly',
                                  'Monthly_Balance']),
                                ('cat_tr',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(drop='if_binary',
                                                                min_frequency=0.05,
                                                                sparse_output=False))]),
                                 ['Occupation', 'Credit_Mix',
                                  'Payment_of_Min_Amount',
                                  'Payment_Behaviour'])])

In [52]:
# import pickle

# ##### EXPORT #######
# url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/preproc_deploy.pkl'
# with open(url, 'wb') as file:
#     pickle.dump(preproc_deploy, file)

In [53]:
# ##### IMPORT #######
# import pickle

# # Caminho do diretório e nome do arquivo
# url = '/home/fernando_cir/code/monrosegregory/credit_score/raw_data/preproc_deploy.pkl'
# with open(url, 'rb') as file:
#     preproc_deploy = preproc_deploy.load(file)